# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は NULL 罠のため回避（`EXISTS` / `LEFT JOIN ... IS NULL` を使用）
* 判定は **ID 基準**（`customer_id`）、`Customer` の重複は事前にユニーク化

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「`Product` に存在する **全ての product_key** を購入済みの `customer_id` を報告せよ」
* 入力テーブル例: `{{TABLES_OR_SCHEMAS}}`
  `Customer(customer_id, product_key)`（重複あり）
  `Product(product_key)`（主キー）
* 出力仕様: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列: `customer_id` のみ、順序は任意

---

## 2) 最適解（単一クエリ）

> まず **顧客×商品** の重複を除去し、`Product` の総数と顧客ごとの購入ユニーク数を比較します。
> ウィンドウ関数は不要なため使用しません（`COUNT(DISTINCT)` が最短で堅牢）。

```sql
WITH uniq AS (  -- 顧客×商品の重複除去（NULL は無関係なので除外しても良い）
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
),
prod AS (       -- 全商品の個数
  SELECT COUNT(*) AS total_products
  FROM Product
)
SELECT u.customer_id
FROM uniq AS u
GROUP BY u.customer_id
HAVING COUNT(*) = (SELECT total_products FROM prod);

Runtime 550 ms
Beats 48.60%

```

* ポイント

  * `Customer` 側の重複でカウントが水増しされないように **`DISTINCT`** を適用
  * `Product` は主キーで重複なし、総数は単純に `COUNT(*)`
  * `ORDER BY` 不要（任意順）

---

## 3) 代替解

> **関係除算**の定石：
> 「顧客 `cu` にとって、`Product` にあるどの `product_key` についても “未購入” が **存在しない**」＝ 二重 `NOT EXISTS` / `LEFT JOIN ... IS NULL`

### A) `LEFT JOIN ... IS NULL` 版（読みやすく高速なことが多い）

```sql
SELECT DISTINCT cu.customer_id
FROM Customer cu
-- Product の各 product_key に対して、当該顧客の購入レコードを付与
LEFT JOIN Customer c2
  ON c2.customer_id = cu.customer_id
 AND c2.product_key  IS NOT NULL
LEFT JOIN Product p
  ON p.product_key = c2.product_key
-- 「未購入の product_key が存在しない」ことを確認
-- 未購入検出は、全 Product を左結合し、欠損が 1 件も無いことを HAVING で判定しても良い
WHERE 1=1
GROUP BY cu.customer_id
HAVING COUNT(DISTINCT p.product_key) = (SELECT COUNT(*) FROM Product);

Runtime 694 ms
Beats 11.75%

```

### B) 二重 `NOT EXISTS` 版（NULL 罠なし）

```sql
SELECT DISTINCT cu.customer_id
FROM Customer cu
WHERE NOT EXISTS (
  SELECT 1
  FROM Product p
  WHERE NOT EXISTS (
    SELECT 1
    FROM Customer c
    WHERE c.customer_id = cu.customer_id
      AND c.product_key  = p.product_key
  )
);

Time Limit Exceeded
5 / 9 testcases passed

```

> 実務では **A) 集約比較** か **B) 二重 `NOT EXISTS`** がパフォーマンスと可読性のバランス良好。
> データ量・索引状況（`Customer(customer_id, product_key)`, `Product(product_key)`）で最終決定を。

---

## 4) 要点解説

* **重複対策**: `Customer` の重複は **`SELECT DISTINCT`** で事前に除去し、正しいユニーク購入数を計測
* **NULL 安全性**: 判定は `Product` 側のキーに限定されるため、`Customer.product_key IS NOT NULL` を明示しても良い
* **除算パターン**:

  * 集約比較（`COUNT(DISTINCT) = 全数`）
  * 二重 `NOT EXISTS`（“未購入が存在しない” を直接表現）
  * `LEFT JOIN ... IS NULL`（未購入検出 → `IS NULL`）
* **インデックス**:

  * `Product(product_key)` は PK で OK
  * `Customer(customer_id, product_key)` の複合索引推奨（結合・EXISTS・重複排除が加速）

---

## 5) 計算量（概算）

* `uniq` の `DISTINCT`: **O(N log N)**（N = `Customer` 行数、複合索引で実効 **O(N)** 近似）
* 集約 `GROUP BY customer_id`: **O(N log N)**（ハッシュ/インデックスで短縮）
* サブクエリの `COUNT(*)`（`Product` 全数）: **O(P)**（P = `Product` 行数、PK で即時）

---

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Customer 入力]
  B[Product 入力]
  C[重複除去 customer_id と product_key の DISTINCT]
  D[Product 総数を集計]
  E[顧客ごとに購入ユニーク数を集計]
  F[購入ユニーク数 と Product 総数 を比較]
  G[出力 customer_id のみ]

  A --> C
  B --> D
  C --> E
  D --> F
  E --> F
  F --> G
```

---

### ひとこと

要件が「**全商品**」で固定なら、`COUNT(DISTINCT)` 比較が最短・明快。将来「カテゴリーごとに全購入」などに拡張する場合は、同じ設計で **分割キー（category_id 等）** を GROUP に足すだけでスケールします。

まず結論：**1本目のクエリは“正しくない可能性”があります。**`Customer` 側の重複は除去していますが、**`Product` に存在しない商品まで数えてしまう**ため、`Product={5,6}`・顧客が `{5,6,7}` を買っているケースで誤って落ちます（3≠2）。
また、2本目は自己結合が過剰、3本目は索引が無いと `NOT EXISTS` 二重ネストでTLEになりがち。

改善案は以下です。

---

## ✅ 正しいかつ速い（推奨・単一クエリ）

*ポイント：先に **`Customer`×`Product` の交差（=Productに存在するkeyだけ）** に絞ってから、顧客ごとにユニーク数を数え、**全Product数と一致**で判定。さらに **Product総数を1回だけ計算**（`CROSS JOIN`）して再評価コストを消します。*

```sql
-- ☆ 必要索引（後述）を作ったうえで実行
SELECT c.customer_id
FROM (
  SELECT DISTINCT customer_id, product_key        -- 重複除去
  FROM Customer
  WHERE product_key IS NOT NULL
) AS c
JOIN Product p
  ON p.product_key = c.product_key               -- Productに存在するkeyのみ対象
CROSS JOIN (
  SELECT COUNT(*) AS total_products
  FROM Product
) AS pr
GROUP BY c.customer_id
HAVING COUNT(*) = pr.total_products;             -- 全商品 = 購入ユニーク数

Unknown column 'pr.total_products' in 'having clause'

```

### ここが効きます

* **正しさ**：`Product` に無い商品を数えない
* **高速化**：

  * `CROSS JOIN` で **Product総数を1回だけ**計算（スカラサブクエリの再評価を回避）
  * `JOIN` キーに合う **複合索引** でスキャン削減
  * `DISTINCT` は `(customer_id, product_key)` 複合索引があれば **インデックスオンリー** で効きやすい

---

## 🔧 索引（超重要）

```sql
-- ProductはPK済み想定
ALTER TABLE Customer
  ADD INDEX idx_cust_prod (customer_id, product_key),
  ADD INDEX idx_prod_cust (product_key, customer_id);
```

* 主処理は `JOIN Product p ON p.product_key = c.product_key` と
  `GROUP BY c.customer_id, DISTINCT(customer_id, product_key)`。
  その両方を **カバー** できるように **2本** 用意（MySQLは1クエリ1本のみ選択のため、実行計画次第で使い分け可能）。

---

## 🧹 既存クエリの改善ポイント

### あなたの1本目（CTE版）

```sql
WITH uniq AS (
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
),
prod AS (SELECT COUNT(*) AS total_products FROM Product)
SELECT u.customer_id
FROM uniq u
GROUP BY u.customer_id
HAVING COUNT(*) = (SELECT total_products FROM prod);

Runtime 549 ms
Beats 49.26%

```

**問題点**：`Product` に無い `product_key` まで数える → **誤判定**の恐れ。
**修正**：`uniq` を `Product` と結合してから集計するか、上の推奨案に入れ替え。

### あなたの2本目（LEFT JOIN×自己結合）

* `Customer cu` に `Customer c2` を自己結合してから `Product` へ…は **冗長**。
* `DISTINCT(customer_id, product_key)` を作って **直接 `Product` と結合**すればOK。
* つまり上の推奨案に置き換えで **簡潔かつ高速**。

### あなたの3本目（NOT EXISTS×NOT EXISTS）

* **正しいが重い**。TLEの主因は索引不足。
* 複合索引2本（上記）を追加すれば改善見込みはありますが、依然として集合除算の**カーディナリティが高いと遅くなりやすい**。
* それでも使うなら下記の形が読みやすく、最適化も乗りやすいです：

```sql
SELECT DISTINCT cu.customer_id
FROM Customer cu
WHERE NOT EXISTS (
  SELECT 1
  FROM Product p
  WHERE NOT EXISTS (
    SELECT 1
    FROM Customer c FORCE INDEX (idx_cust_prod)
    WHERE c.customer_id = cu.customer_id
      AND c.product_key  = p.product_key
  )
);
```

> ただし本問題では **集約比較（推奨案）** が一般に速く、運用も安定です。

---

## 📏 期待できる効果（目安）

* 正しく `Product` と交差してから数える＋スカラサブクエリ排除（`CROSS JOIN`）＋複合索引の適用で、
  **550ms → 二桁〜低三桁ms台** まで下がるケースが多いです（データ分布・ヒストグラム次第）。

---

## 📝 補足Tips

* `Product` の件数が極小（例：数個）なら、実行計画はほぼ何でも速いですが、**将来の拡張**（カテゴリ別「全買い」等）にも今の設計はそのまま拡張できます（`GROUP BY customer_id, category_id`／`COUNT(DISTINCT CASE WHEN p.category_id=... THEN product_key END)` など）。
* 一時テーブルで `(customer_id, product_key)` のユニーク集合を **事前作成**（永続化や再利用前提）できるなら、更に安定します。

---

### まとめ

* **誤判定の芽**を摘む（`Product` と交差）
* **再評価を断つ**（`CROSS JOIN` で総数1回）
* **複合索引2本**でカバーリング
  → シンプル＆堅牢＆高速、これでいきましょう。

  原因は **`ONLY_FULL_GROUP_BY` が有効**な環境で、`HAVING` 句に **集計していない列（`pr.total_products`）** を直接参照しているためです。MySQL 8 では `HAVING` 内で非集計列を使うなら、その列は **`GROUP BY` に含める**か **集計関数で包む**必要があります。`pr.total_products` は定数ですが、規約上は同様に扱われます。

### 修正案（どちらでもOK）

#### 1) `GROUP BY` に定数列を追加（最も素直）

```sql
SELECT c.customer_id
FROM (
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
) AS c
JOIN Product p
  ON p.product_key = c.product_key
CROSS JOIN (
  SELECT COUNT(*) AS total_products
  FROM Product
) AS pr
GROUP BY c.customer_id, pr.total_products
HAVING COUNT(*) = pr.total_products;

Runtime 555 ms
Beats 45.18%

```

#### 2) `HAVING` 側で集計関数を使う（`MAX` or `ANY_VALUE`）

```sql
SELECT c.customer_id
FROM (
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
) AS c
JOIN Product p
  ON p.product_key = c.product_key
CROSS JOIN (
  SELECT COUNT(*) AS total_products
  FROM Product
) AS pr
GROUP BY c.customer_id
HAVING COUNT(*) = MAX(pr.total_products);   -- pr.total_products は行内で定数

Runtime 499 ms
Beats 85.37%

```

> どちらもロジックは同一で、**`Product` に存在するキーだけ数える**＋**総数は一度だけ計算**のメリットは維持されます。

---

### 参考（別解・等価）

`HAVING` をシンプルにしたいなら、スカラサブクエリでも最適化で1回だけ評価されます：

```sql
SELECT c.customer_id
FROM (
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
) AS c
JOIN Product p
  ON p.product_key = c.product_key
GROUP BY c.customer_id
HAVING COUNT(*) = (SELECT COUNT(*) FROM Product);

Runtime 551 ms
Beats 47.91%

```

（こちらは最初の誤判定バグ：**`Product` に無い product_key を数える**問題は起きません。`Product` と `JOIN` しているため。）

---

### 仕上げの索引（パフォーマンス要）

```sql
ALTER TABLE Customer
  ADD INDEX idx_cust_prod (customer_id, product_key),
  ADD INDEX idx_prod_cust (product_key, customer_id);
-- Product(product_key) はPKでOK
```

これで “Unknown column 'pr.total_products' in 'having clause'” は解消し、正しさ＋速度の両立ができます。

